In [60]:
# import libraries
import sqlite3 as sl
import pandas as pd
import numpy as np

import warnings
warnings.filterwarnings('ignore')

import scipy.stats as scs
import statsmodels.api as sm
from statsmodels.formula.api import ols
from statsmodels.stats.multicomp import pairwise_tukeyhsd, MultiComparison

import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px 
import plotly.figure_factory as ff

In [61]:
def get_db_connn(path_to_db):
    conn = sl.connect(path_to_db)
    return conn


def get_table_names(conn):
    query = """
            select name from sqlite_master where type='table';
            """
    res = conn.execute(query).fetchall()
    table_names = [r[0] for r in res]
    return table_names


def get_table_column_names(conn, table_name):
    query = f"""
            pragma table_info({table_name})
            """
    res = conn.execute(query).fetchall()
    col_names = [r[1] for r in res]
    return col_names


def load_table_as_df(conn, table_name, to_display=True):
    query = f"""
            select * from {table_name}
            """
    df = pd.read_sql(query, conn)
    if to_display:
        display(df.head())
    return df

In [62]:
db_path = '../data/northWind.sqlite'
conn = get_db_connn(db_path)

# Question 1: Is there a difference in quantity of products sold by discount rate? 

In [65]:
# obtain necessary data

#get the data for order quantities and discounts
orderdetails_q = """select * from OrderDetail;"""
orderdetails_df = pd.read_sql(orderdetails_q, conn)
orderdetails_df.head()

,Id,OrderId,ProductId,UnitPrice,Quantity,Discount
0,10248/11,10248,11,14.0,12,0.0
1,10248/42,10248,42,9.8,10,0.0
2,10248/72,10248,72,34.8,5,0.0
3,10249/14,10249,14,18.6,9,0.0
4,10249/51,10249,51,42.4,40,0.0


In [66]:
print(np.sort(orderdetails_df['Discount'].unique()))

[0.   0.01 0.02 0.03 0.04 0.05 0.06 0.1  0.15 0.2  0.25]


There are a few discounts between zero and 5%, and 1 between 5% and 10%. Suggest we use the follow discount categories to do Order group by: 0%, 5% or less, between 5 %-10%, 15%, 20%, 25%

Will begin by simplifying: **control**: No Discount for Product **treatment**: Some Discount (no matter what amount) for Product


In [67]:
#create a new column, has_discount, containing 0 or 1
orderdetails_df['has_discount'] = orderdetails_df.apply(lambda row: 1 if row.Discount> 0.0 else 0, axis=1)
orderdetails_df.head()

,Id,OrderId,ProductId,UnitPrice,Quantity,Discount,has_discount
0,10248/11,10248,11,14.0,12,0.0,0
1,10248/42,10248,42,9.8,10,0.0,0
2,10248/72,10248,72,34.8,5,0.0,0
3,10249/14,10249,14,18.6,9,0.0,0
4,10249/51,10249,51,42.4,40,0.0,0


In [68]:
# Cool...now we know which products were discounted on not
# we are really only interested in Quantity and has_discount so drop what we do not need.
quantity_discount_df = orderdetails_df.drop(columns=['Id', 'OrderId', 'UnitPrice', 'Discount'])
quantity_discount_df.head()

,ProductId,Quantity,has_discount
0,11,12,0
1,42,10,0
2,72,5,0
3,14,9,0
4,51,40,0


## {Mean Quantity Sold between Products w/ Discount and Products w/out Discount}
H0: There is no difference in mean quantity between products with a discount and products without a discount. 

HA: There is a difference in mean quantity between products with a discount and products without a discount.

In [69]:
# perform test

discount_quantity = quantity_discount_df.loc[quantity_discount_df['has_discount']==1, 'Quantity']
nodiscount_quantity = quantity_discount_df.loc[quantity_discount_df['has_discount']==0, 'Quantity']

anova_result = scs.f_oneway(discount_quantity, nodiscount_quantity)
print(anova_result)
anova_result.pvalue

F_onewayResult(statistic=41.971781088386834, pvalue=1.1440924523215329e-10)


1.1440924523215329e-10

In [80]:
#Tukey Testing not really needed since we did a comparison between ONLY 2 groups but I like the simple_table output
multicompare_discount = MultiComparison(quantity_discount_df['Quantity'], quantity_discount_df['has_discount'])

In [81]:
tukey_hsd = multicompare_discount.tukeyhsd(alpha=.05)

In [82]:
simple_table = tukey_hsd.summary()
simple_table

group1,group2,meandiff,p-adj,lower,upper,reject
0,1,5.3945,0.001,3.7616,7.0274,True


In [79]:
# What happens when we just use all the discount amounts?
multicompare_discount_for_real = MultiComparison(orderdetails_df['Quantity'], orderdetails_df['Discount'])
tukey_hsd_for_real = multicompare_discount_for_real.tukeyhsd(alpha=.05)
summary = tukey_hsd_for_real.summary()
summary

group1,group2,meandiff,p-adj,lower,upper,reject
0.0,0.01,-19.7153,0.9,-80.3306,40.9001,False
0.0,0.02,-19.7153,0.9,-62.593,23.1625,False
0.0,0.03,-20.0486,0.725,-55.0714,14.9742,False
0.0,0.04,-20.7153,0.9,-81.3306,39.9001,False
0.0,0.05,6.2955,0.0011,1.5381,11.053,True
0.0,0.06,-19.7153,0.9,-80.3306,40.9001,False
0.0,0.1,3.5217,0.4269,-1.3783,8.4217,False
0.0,0.15,6.6669,0.0014,1.551,11.7828,True
0.0,0.2,5.3096,0.0303,0.2508,10.3684,True
0.0,0.25,6.525,0.0023,1.3647,11.6852,True


## Conclusion

## EffectSize/PowerAnalysis/Tukey Testing (if needed)

# Question 2 - Is there a difference in quantity of orders between countries

In [ ]:
# obtain necessary data

# Plan was to get all OrderDetails, join in ShipCountry and then
# Groupby the countries, and sum the quantities then test it out.
# Data EDA/Joining proved challenging for the allocated time. 

## {Put test name here}
H0: 

HA:

In [ ]:
# perform test

## Conclusion

# Question 3: {You decide}

In [ ]:
# obtain necessary data

## {Put test name here}
H0: 

HA:

In [ ]:
# perform test

## Conclusion

## EffectSize/PowerAnalysis/Tukey Testing (if needed)